Load packages and init SQLContext:

In [1]:
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
sqlContext = SQLContext(sc)

In [2]:
dataset = sqlContext.read.json("file:///home/bdm/twitter.small")

Parse dates before we start processing:

In [3]:
from datetime import datetime
from operator import add

def parse_date(date_field):
    date_parts = date_field.split()
    date_str = "%s/%s/%s:%s" % (date_parts[2],
                                date_parts[1],
                                date_parts[5],
                                date_parts[3])
    return datetime.strptime(date_str, '%d/%b/%Y:%H:%M:%S')

Register temp table:

In [4]:
dataset.registerTempTable("tweets")

# Tweets-oriented

Tweets per day, month, year

In [5]:
tweets_per_day = dataset.map(lambda x: (parse_date(x.user.created_at).strftime("%Y-%m-%d"), 1)).reduceByKey(add)
countDF = sqlContext.createDataFrame(tweets_per_day).withColumnRenamed("_1","date").withColumnRenamed("_2","count")
countDF.registerTempTable("tweetsCount")
sqlContext.sql("SELECT * from tweetsCount").show()

+----------+-----+
|      date|count|
+----------+-----+
|2012-11-16|    1|
|2012-11-12|    1|
|2011-01-18|    2|
|2008-08-13|    1|
|2010-11-30|    1|
|2009-09-04|    1|
|2010-04-03|    1|
|2012-10-15|    1|
|2012-03-13|    1|
|2009-11-25|    1|
|2010-11-03|    1|
|2008-04-13|    1|
|2010-11-25|    1|
|2008-06-19|    1|
|2009-06-12|    1|
|2010-02-03|    1|
|2009-02-21|    1|
|2009-12-15|    1|
|2009-10-08|    1|
|2010-10-22|    1|
+----------+-----+
only showing top 20 rows



Interactions per day, month, year

In [6]:
interactions_per_day = dataset.map(lambda x: (parse_date(x.user.created_at).strftime("%Y-%m-%d")\
                                              , x.favorite_count + x.retweet_count)).reduceByKey(add)
interactionsDF = sqlContext.createDataFrame(interactions_per_day).withColumnRenamed("_1","date")\
    .withColumnRenamed("_2","interactions") 
interactionsDF.registerTempTable("tweetInteractions")
sqlContext.sql("SELECT * from tweetInteractions").show()

+----------+------------+
|      date|interactions|
+----------+------------+
|2012-11-16|           0|
|2012-11-12|           1|
|2011-01-18|           0|
|2008-08-13|           0|
|2010-11-30|           0|
|2009-09-04|           0|
|2010-04-03|           0|
|2012-10-15|           0|
|2012-03-13|           1|
|2009-11-25|           0|
|2010-11-03|           0|
|2008-04-13|           0|
|2010-11-25|           0|
|2008-06-19|           0|
|2009-06-12|           0|
|2010-02-03|           0|
|2009-02-21|           0|
|2009-12-15|           0|
|2009-10-08|           0|
|2010-10-22|           0|
+----------+------------+
only showing top 20 rows



## Join Two Dataframes

In [ ]:
userTweetsDF = sqlContext.sql("""SELECT t1.date,
           t1.count,
           t2.interactions 
      FROM tweetInteractions t2
 LEFT JOIN tweetsCount t1 ON t2.date = t1.date
UNION
    SELECT t2.date,
           t1.count,
           t2.interactions
      FROM tweetInteractions t2
RIGHT JOIN tweetsCount t1 ON t2.date = t1.date""")
userTweetsDF.show()

## Create Table in Cassandra

#### Create Keyspace and Table in Cassandra
  
CREATE KEYSPACE assignment
WITH REPLICATION = { ’class’ : ’SimpleStrategy’, ’replication_factor’ : 1 };

CREATE TABLE tweetInteractions (  
date date PRIMARY KEY,  
count int,  
interactions int  
);

#### Write To Cassandra

In [ ]:
userTweetsDF.select("date", "count", "interactions")\  
.write.format("org.apache.spark.sql.cassandra")\  
.options(table="userTweets", keyspace="assignment")\  
.save(mode="append") 

#### Read from Cassandra

In [ ]:
userTweets = sql.read.format("org.apache.spark.sql.cassandra")\
.load(keyspace="assignment", table="userTweets")

userTweets.registerTempTable("userTweets")

## Month With The Most Interactions

In [ ]:
sqlContext.sql(
  """SELECT YEAR(date) AS Year , month(date) AS Month, SUM(interactions) AS Interactions
     FROM userTweets
     GROUP BY Year(date) , Month(date)
     ORDER BY Interactions DESC
     LIMIT 1
     """).show()

# Movies-oriented

## Tweets per movie per day

In [8]:
movies_tweets_per_day = dataset.map(lambda x: ((x.entities.urls[0].display_url, parse_date(x.user.created_at).strftime("%Y-%m-%d")), 1)).reduceByKey(add)
movieCountsDF = sqlContext.createDataFrame(movies_tweets_per_day.map(lambda tup: (tup[0][0], tup[0][1], tup[1])))\
        .withColumnRenamed("_2","date").withColumnRenamed("_3","count").withColumnRenamed("_1","title")
movieCountsDF.show()

+--------------------+----------+-----+
|               title|      date|count|
+--------------------+----------+-----+
|imdb.com/title/tt...|2009-11-01|    1|
|imdb.com/title/tt...|2008-04-13|    1|
|imdb.com/title/tt...|2010-02-24|    1|
|imdb.com/title/tt...|2009-06-29|    1|
|imdb.com/title/tt...|2009-03-17|    1|
|imdb.com/title/tt...|2009-06-02|    1|
|imdb.com/title/tt...|2010-03-20|    1|
|imdb.com/title/tt...|2010-11-06|    1|
|imdb.com/title/tt...|2009-06-09|    1|
|imdb.com/title/tt...|2009-03-17|    1|
|imdb.com/title/tt...|2011-01-25|    1|
|imdb.com/title/tt...|2009-12-15|    1|
|imdb.com/title/tt...|2009-02-21|    1|
|imdb.com/title/tt...|2011-02-21|    1|
|imdb.com/title/tt...|2012-11-12|    1|
|imdb.com/title/tt...|2009-06-12|    1|
|imdb.com/title/tt...|2008-06-19|    1|
|imdb.com/title/tt...|2009-03-17|    1|
|imdb.com/title/tt...|2010-09-14|    1|
|imdb.com/title/tt...|2012-10-15|    1|
+--------------------+----------+-----+
only showing top 20 rows



#### Create Table in Cassandra

CREATE TABLE movieCounts (  
title text,  
date date,  
counts int,  
primary key(title, date)  
);

#### Write To Cassandra

In [ ]:
movieCountsDF.select("title", "date", "counts")\  
.write.format("org.apache.spark.sql.cassandra")\  
.options(table="movieCounts", keyspace="assignment")\  
.save(mode="append") 

#### Read from Cassandra

In [ ]:
movieCounts = sql.read.format("org.apache.spark.sql.cassandra")\
.load(keyspace="assignment", table="movieCounts")

movieCounts.registerTempTable("popularity")

## 20 Most Popular Movies

In [ ]:
sqlContext.sql("SELECT title,count from popularity ORDER BY count DESC").show()

## Engagement per movie

In [9]:
movies_engagement = sqlContext.sql("select entities.urls[0].display_url as movie, "+\
               "sum(favorite_count) + sum(retweet_count) as engagement "
               "from tweets "+\
               "group by entities.urls[0].display_url")

movies_engagement.show()

+--------------------+----------+
|               movie|engagement|
+--------------------+----------+
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         1|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
|imdb.com/title/tt...|         0|
+--------------------+----------+
only showing top 20 rows



#### Create Table in Cassandra

In [ ]:
CREATE TABLE movieEngagements (
movie text PRIMARY KEY,
engagements int
);

#### Write To Cassandra

In [ ]:
movie_engagements.select("movie", "engagement")\  
.write.format("org.apache.spark.sql.cassandra")\  
.options(table="movieEngagements", keyspace="assignment")\  
.save(mode="append") 

#### Read from Cassandra

In [ ]:
movieEngagements = sql.read.format("org.apache.spark.sql.cassandra")\
.load(keyspace="assignment", table="movieEngagements")

## Popularity per movie per language:

In [10]:
movies_language_pop = sqlContext.sql("select entities.urls[0].display_url as movie, "+\
               "lang as language, count(1) as pop "
               "from tweets "+\
               "group by entities.urls[0].display_url, lang")



movies_language_pop.show()

+--------------------+--------+---+
|               movie|language|pop|
+--------------------+--------+---+
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      da|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      sk|  1|
|imdb.com/title/tt...|      en|  1|
|imdb.com/title/tt...|      en|  1|
+--------------------+--------+---+
only showing top 20 rows



## Create Table in Cassandra

CREATE TABLE moviePopularity (  
movie text PRIMARY KEY,  
language varchar(2),  
pop int  
);

## Write To Cassandra

In [ ]:
movie_languages_pop.select("movie", "language", "pop")\  
.write.format("org.apache.spark.sql.cassandra")\  
.options(table="moviePopularity", keyspace="assignment")\  
.save(mode="append") 

## Read from Cassandra

In [ ]:
moviePopularity = sql.read.format("org.apache.spark.sql.cassandra")\
.load(keyspace="assignment", table="moviePopularity")

## Film Most Tweeted About in Spanish

In [ ]:
moviePopularity.registerTempTable("lang")
sqlContext.sql("""
SELECT movie, language, SUM(pop) AS count FROM lang
WHERE language = 'en'
GROUP BY movie, language
ORDER BY count DESC
LIMIT 1
""").show()

# Users-oriented

Number of followers, favourites, statuses and listings per user, at different time points:

In [11]:
user_stats = dataset.map(lambda x: (x.user.screen_name, \
    x.created_at, x.user.followers_count, x.user.favourites_count, x.user.statuses_count, x.user.listed_count))
usersDF = sqlContext.createDataFrame(user_stats).withColumnRenamed("_1","username")\
    .withColumnRenamed("_2","createdAt") \
    .withColumnRenamed("_4","favs")\
    .withColumnRenamed("_5","statuses") \
    .withColumnRenamed("_3","followers")\
    .withColumnRenamed("_6","listed") 

usersDF.show()

+---------------+--------------------+---------+----+--------+------+
|       username|           createdAt|followers|favs|statuses|listed|
+---------------+--------------------+---------+----+--------+------+
|     Nat_ta_gun|Thu Feb 28 14:43:...|      114| 679|   47133|     2|
|   Carterwade99|Thu Feb 28 14:47:...|      151| 121|    9281|     0|
|       MircheBg|Thu Feb 28 14:58:...|      201| 333|    1410|     6|
|     zoltanmora|Thu Feb 28 15:00:...|       81|  23|     853|     1|
|     zoltanmora|Thu Feb 28 15:04:...|       81|  23|     853|     1|
|        yenda_m|Thu Feb 28 15:05:...|       75|   0|     655|     1|
|     zoltanmora|Thu Feb 28 15:07:...|       81|  23|     853|     1|
|        BaderJr|Thu Feb 28 15:08:...|      137|  76|    8231|     0|
|      paulpasia|Thu Feb 28 15:09:...|      215|  15|    1091|     4|
|         MMTA7W|Thu Feb 28 15:26:...|      130| 135|   22372|     0|
|         MMTA7W|Thu Feb 28 15:27:...|      130| 135|   22372|     0|
|         MMTA7W|Thu

## Create Table in Cassandra

CREATE TABLE users (  
username text,  
createdAt varchar(30),  
followers int,  
favs int,  
statuses int,  
listed int,  
primary key (username, createdAt)  
);

## Write To Cassandra

In [ ]:
usersDF.select("username", "createdAt", "favs", "statuses", "followers", "listed")\  
.write.format("org.apache.spark.sql.cassandra")\  
.options(table="users", keyspace="assignment")\  
.save(mode="append") 

## Read from Cassandra

In [ ]:
users = sql.read.format("org.apache.spark.sql.cassandra")\
.load(keyspace="assignment", table="users")

users.registerTemporaryTable("users")

### Latest update of the user with the most followers

In [ ]:
sqlContext.sql("""select u.*
from users u
Join
(
    select  t.Username, max(followers) AS maximum
    from   users t
    group by t.Username
    order by maximum desc limit 1
)   p
On u.Username = p.Username
Order By u.Date DESC
Limit 1""").show()